# Homework 8

> I pledge my Honor that I have abided by the Stevens Honor System. - Joshua Schmidt

### Part 0

spark 1 example code

imports

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "hw8")

In [6]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [3]:
# load data
data = sc.textFile('re_u.data')
data.count()

100000

In [4]:
data.take(5)

['196,242,3', '186,302,3', '22,377,1', '244,51,2', '166,346,1']

In [7]:
ratings = data.map(lambda line: line.split(',')).map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])))
ratings.count()

100000

In [8]:
ratings.take(5)

[Rating(user=196, product=242, rating=3.0),
 Rating(user=186, product=302, rating=3.0),
 Rating(user=22, product=377, rating=1.0),
 Rating(user=244, product=51, rating=2.0),
 Rating(user=166, product=346, rating=1.0)]

In [9]:
# alternating least squares
rank = 10
num_iter = 10
model = ALS.train(ratings, rank, num_iter)

In [10]:
test_data = ratings.map(lambda elem: (elem[0], elem[1]))
predictions = model.predictAll(test_data).map(lambda pred: ((pred[0], pred[1]), pred[2]))
ratings_predictions = ratings.map(lambda rating: ((rating[0], rating[1]), rating[2])).join(predictions)
mse = ratings_predictions.map(lambda elem: (elem[1][0] - elem[1][1]) ** 2).mean()
print(f'mean squared error: {mse:.3f}')

mean squared error: 0.483


### Part 1

final project recommendation code

imports

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [26]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("hw8") \
    .getOrCreate()

In [23]:
training = spark.read.csv("trainItem.data", header = False)
training.show(5)

+------+------+---+
|   _c0|   _c1|_c2|
+------+------+---+
|199808|248969| 90|
|199808|  2663| 90|
|199808| 28341| 90|
|199808| 42563| 90|
|199808| 59092| 90|
+------+------+---+
only showing top 5 rows



In [24]:
training = training.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
training.show(5)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|    90|
|199808|  2663|    90|
|199808| 28341|    90|
|199808| 42563|    90|
|199808| 59092|    90|
+------+------+------+
only showing top 5 rows



In [27]:
from pyspark.sql.types import IntegerType
training = training.withColumn("userID", training["userID"].cast(IntegerType()))
training = training.withColumn("itemID", training["itemID"].cast(IntegerType()))
training = training.withColumn("rating", training["rating"].cast('float'))
training.show(3)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|  90.0|
|199808|  2663|  90.0|
|199808| 28341|  90.0|
+------+------+------+
only showing top 3 rows



In [28]:
# Create ALS model
als = ALS(
    maxIter=5,
    rank = 5,
    regParam=0.01,
    userCol="userID",
    itemCol="itemID",
    ratingCol="rating",
    nonnegative = True,
    implicitPrefs = False,
    coldStartStrategy="drop"
)

In [29]:
model = als.fit(training)

In [30]:
testing = spark.read.csv("testItem.data", header = False)

In [31]:
testing = testing.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
testing.show(5)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199810|208019|     0|
|199810| 74139|     0|
|199810|  9903|     0|
|199810|242681|     0|
|199810| 18515|     0|
+------+------+------+
only showing top 5 rows



In [32]:
testing = testing.withColumn("userID", testing["userID"].cast(IntegerType()))
testing = testing.withColumn("itemID", testing["itemID"].cast(IntegerType()))
testing = testing.withColumn("rating", testing["rating"].cast('float'))
testing.show(3)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199810|208019|   0.0|
|199810| 74139|   0.0|
|199810|  9903|   0.0|
+------+------+------+
only showing top 3 rows



In [33]:
predictions = model.transform(testing)
predictions.show(5)

+------+------+------+----------+
|userID|itemID|rating|prediction|
+------+------+------+----------+
|230073|   463|   0.0|113.081635|
|230962|   471|   0.0| 84.517975|
|218845|  1088|   0.0| 53.643383|
|209697|  1088|   0.0| 21.104813|
|224445|  2142|   0.0| 33.803844|
+------+------+------+----------+
only showing top 5 rows



In [34]:
# save dataframe into a folder 'predictions' 
# with a single file (coalesce(1))
# but you can not assign the filename
predictions.coalesce(1).write.csv("predictions")

In [36]:
# save dataframe to a single csv file 
df = predictions.toPandas()
print(df.head())
df.to_csv('myprediction.csv')

   userID  itemID  rating  prediction
0  230073     463     0.0  113.081635
1  230962     471     0.0   84.517975
2  218845    1088     0.0   53.643383
3  209697    1088     0.0   21.104813
4  224445    2142     0.0   33.803844


### Part 2

In [49]:
from pyspark.mllib.recommendation import ALS
# load data
data = sc.textFile('re_u.data')
data.count()

100000

In [45]:
data.take(5)

['196,242,3', '186,302,3', '22,377,1', '244,51,2', '166,346,1']

In [46]:
ratings = data.map(lambda line: line.split(',')).map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])))
ratings.count()

100000

In [47]:
ratings.take(5)

[Rating(user=196, product=242, rating=3.0),
 Rating(user=186, product=302, rating=3.0),
 Rating(user=22, product=377, rating=1.0),
 Rating(user=244, product=51, rating=2.0),
 Rating(user=166, product=346, rating=1.0)]

In [51]:
# different ranks
for rank in [5, 7, 10, 20]:
    num_iter = 20
    curr_model = ALS.train(ratings, rank, num_iter)
    test_data = ratings.map(lambda elem: (elem[0], elem[1]))
    predictions = curr_model.predictAll(test_data).map(lambda pred: ((pred[0], pred[1]), pred[2]))
    ratings_predictions = ratings.map(lambda rating: ((rating[0], rating[1]), rating[2])).join(predictions)
    mse = ratings_predictions.map(lambda elem: (elem[1][0] - elem[1][1]) ** 2).mean()
    print(f'mean squared error for rank {rank}: {mse:.3f}')

mean squared error for rank 5: 0.615
mean squared error for rank 7: 0.547
mean squared error for rank 10: 0.474
mean squared error for rank 20: 0.293


In [52]:
# different num iterations
for num_iter in [2, 5,10, 20]:
    rank = 20
    curr_model = ALS.train(ratings, rank, num_iter)
    test_data = ratings.map(lambda elem: (elem[0], elem[1]))
    predictions = curr_model.predictAll(test_data).map(lambda pred: ((pred[0], pred[1]), pred[2]))
    ratings_predictions = ratings.map(lambda rating: ((rating[0], rating[1]), rating[2])).join(predictions)
    mse = ratings_predictions.map(lambda elem: (elem[1][0] - elem[1][1]) ** 2).mean()
    print(f'mean squared error for num_iter {num_iter}: {mse:.3f}')

mean squared error for num_iter 2: 0.481
mean squared error for num_iter 5: 0.337
mean squared error for num_iter 10: 0.308
mean squared error for num_iter 20: 0.291


In [68]:
# different num iterations
import pyspark
for data_size in [2000, 5000, 10000, 20000, 50000, 100000]:
    rank = 20
    num_iter = 20
    curr_ratings = sc.parallelize(ratings.take(data_size))
    curr_model = ALS.train(curr_ratings, rank, num_iter)
    test_data = curr_ratings.map(lambda elem: (elem[0], elem[1]))
    predictions = curr_model.predictAll(test_data).map(lambda pred: ((pred[0], pred[1]), pred[2]))
    ratings_predictions = curr_ratings.map(lambda rating: ((rating[0], rating[1]), rating[2])).join(predictions)
    mse = ratings_predictions.map(lambda elem: (elem[1][0] - elem[1][1]) ** 2).mean()
    print(f'mean squared error for num_iter {num_iter}: {mse:.3f}')

mean squared error for num_iter 20: 0.000
mean squared error for num_iter 20: 0.001
mean squared error for num_iter 20: 0.003
mean squared error for num_iter 20: 0.026
mean squared error for num_iter 20: 0.152
mean squared error for num_iter 20: 0.292


For the above 3 different scenarios, from your observation, how MSE is changed related to the parameters? That is, which factor, the rank size, or the numIterations size, or the data size will change the MSE value more significantly?

As rank increases from 5 to 20, mean squared error decreases. As the number of iterations increases, all else being equal, the mean squared error decreases. As the amount of data used for training and testing increases, the mean squared error increases. Data size will change the MSE value most significantly, because as the amount of training and testing data increases, there will be more outliers that will contribute to a higher MSE. The rank size seems to have the second highest effect on the MSE, with a range of 0.615 to 0.293 vs 0.481 to 0.291 for number of iterations.